In [ ]:
import sys
print(sys.version)

Dependencies

In [2]:
import time
import board
import busio
import Jetson.GPIO as GPIO
import sys
import os

#Display clontrol library
from IPython import display
from ipywidgets import Button, HBox, VBox, Layout, Label, IntSlider
import ipywidgets as widgets
from IPython.display import display, clear_output

#PWM driver dilivery
from pca9685_driver import Device

servo = Device(0x40,1)
servo.set_pwm_frequency(50)

time.sleep(0.5)

#control steering position fouction
# Lateral control
servo.set_pwm(0,307)
#range 245x370 (307 to center) (245 all to the right) (370 all to the left)

#control motor power
# Longitudinal control
servo.set_pwm(1,330)
#to stop 330
#to go forward 345
#to go backward 320 to times

Control

In [23]:
# lateral control with buttons

# left
button1 = Button(description='left',
           layout=Layout(width='33%', height='80px'))
# s_forward
button2 = Button(description='s_forward',
           layout=Layout(width='33%', height='80px'))
# right
button3 = Button(description='right',
           layout=Layout(width='33%', height='80px'))

hbox1 = HBox([button1,button2,button3])

# lateral control with slider

slider1 = widgets.FloatSlider(
            value=307.0,
            min=245.0,
            max=370.0,
            step=1.0,
            description='steering:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )

Lateral_control_display = VBox([Label('Lateral Control:'), slider1, hbox1])# create the Lateral_control_display


# longitudinal control

# reverse
button4 = Button(description='reverse',
           layout=Layout(width='33%', height='80px'))
# stop
button5 = Button(description='stop',
            button_style= 'danger',
            layout=Layout(width='33%', height='80px'))
# p_forward
button6 = Button(description='p_forward',
           layout=Layout(width='33%', height='80px'))

hbox2 = HBox([button4,button5,button6])

# longitudinal control with slider max=350
slider2 = widgets.FloatSlider(
            value=330.0,
            min=320.0,
            max=350.0,
            step=1.0,
            description='power:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f'
        )

Longitudinal_control_display = VBox([Label('Longitudinal Control:'), slider2, hbox2])# create the Lateral_control_display

output = widgets.Output()

#clear output button
button7 = Button(description='clear',
           layout=Layout(width='10%', height='145px'))

hbox3 = HBox([Lateral_control_display,Longitudinal_control_display,button7,output]) 

# End display all
display(hbox3)# End display all

# Now all the functions to control

# steering functions

def on_button1_clicked(b):
    with output:
        value = 370
        print("left:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button1.on_click(on_button1_clicked)

def on_button2_clicked(b):
    with output:
        value = 307
        print("s_forward:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button2.on_click(on_button2_clicked)

def on_button3_clicked(b):
    with output:
        value = 245
        print("right:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(0,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button3.on_click(on_button3_clicked)

def on_slider1_change(change):
    with output:
        print("steering:",change['new'])
        servo.set_pwm(0,((int)(change['new'])))

slider1.observe(on_slider1_change, names='value')


#control motor power functions

# reverse button control
def on_button4_clicked(b):
    with output:
        value = 320
        print("reverse:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button4.on_click(on_button4_clicked)

#stop button function
def on_button5_clicked(b):
    with output:
        value = 330
        print("stop:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)

button5.on_click(on_button5_clicked)

# power button control function
def on_button6_clicked(b):
    with output:
        value = 345
        print("p_forward:",value)
        #control steering position fouction
        # Lateral control
        servo.set_pwm(1,value)
        #range 245x370 (307 to center) (245 all to the right) (370 all to the left)
        
button6.on_click(on_button6_clicked)

# power control slider
def on_slider2_change(change):
    with output:
        print("power:",change['new'])
        servo.set_pwm(1,((int)(change['new'])))

slider2.observe(on_slider2_change, names='value')

#clear output funcition
def on_button7_clicked(b):
    output.clear_output()
    pass

button7.on_click(on_button7_clicked)